In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec            # to arrange the plots nicely
import pandas as pd

%matplotlib inline

In [3]:
#==============================================================
# Useful aliases - Will be filled when reading excel file
#==============================================================
fixed_pmt_name = '' ###  name of pmt with FIXED HV
variable_pmt_name = '' ###  name of pmt with VARIABLE HV
time = 0 ###  time window
HV_fixed = 0 ###  HV value fixed

deriv_coin = [] ### list of derivatives of coincidences plot
deriv_err = [] ### list of errors on derivatives
mean_err = 0 ### mean of deriv_err
low = 0 ### lower bound of plateau
up = 1000 ### upper bound of plateau
med = 0 ### central value of plateau

#==================================== 
# Compute errors on measurements
#==================================== 
#===== Systematic uncertainty on HV applied
# Check in the Power Supply manual the precision on the HV value shown in the front panel of the Power Supply module
# Define function to compute the error on HV
def error_on_HV(HV):
    a = 0.01
    b = 5
    return HV*a+b


def error_on_threshold(thr):
    # error on threshold set?
    return 0

#===== Statistical uncertainty on counts
# Poisson uncertainty on counts
def error_on_counts(n):
    return np.sqrt(n)


#==================================== 
# Routine to find plateau [CURVA DI LAVORO]
#==================================== 
def find_plateau_hv(data, tolerance=1):
    ### tolerance: scale factor on mean error (to select range for plateau)

    deriv_coin = [] ### list of derivatives of coincidences plot
    deriv_err = [] ### list of errors on derivatives
    mean_err = 0 ### mean of deriv_err
    low = 0 ### lower bound of plateau
    up = 1000 ### upper bound of plateau
    med = 0 ### central value of plateau
    
    #=============== Compute derivatives
    for i in range(len(data)):
        #-------- Skip first point
        if i==0:
            d = 0
            err = 0
        #-------- Compute slope point-by-point
        else:
            d = (data.coin[i] - data.coin[i-1]) / (data.HV[i] - data.HV[i-1])
            err = np.sqrt(data.coin_err[i]**2 + data.coin_err[i-1]**2) / (data.HV[i] - data.HV[i-1])
        deriv_err.append(err)
        deriv_coin.append(d) 
    
    # compute mean value of error on derivatives
    mean_err = np.mean(deriv_err)
    
    #=============== Find plateau range 
    for j in range(len(data)):
        n = len(data) - 1
        #print(j,n-j)
        if ( (deriv_coin[j] > - tolerance*mean_err) & (deriv_coin[j] < tolerance*mean_err) & (low == 0) & (j > 0) ):
            low = data.HV[j-1]
            #print('low', low)
        if ( (deriv_coin[n-j] > -tolerance*mean_err) & (deriv_coin[n-j] < tolerance*mean_err) & (up == 1000) ):
            up = data.HV[n-j]
            #print('up', up)
    med = (low+up) / 2

    return  deriv_coin, deriv_err, mean_err, low, up, med



#==================================== 
# Routine to find plateau [DISCRIMINATION]
#==================================== 
def find_plateau_discr(data, tolerance=1):
    ### tolerance: scale factor on mean error (to select range for plateau)
    
    deriv_coin = [] ### list of derivatives of coincidences plot
    deriv_err = [] ### list of errors on derivatives
    mean_err = 0 ### mean of deriv_err
    low = 0 ### lower bound of plateau
    up = 1000 ### upper bound of plateau
    med = 0 ### central value of plateau
    
    #=============== Compute derivatives
    for i in range(len(data)):
        #-------- Skip first point
        if i==0:
            d = 0
            err = 0
        #-------- Compute slope point-by-point
        else:
            d = (data.coin[i] - data.coin[i-1]) / (data.threshold[i] - data.threshold[i-1])
            err = np.sqrt(data.coin_err[i]**2 + data.coin_err[i-1]**2) / (data.threshold[i] - data.threshold[i-1])
        deriv_err.append(err)
        deriv_coin.append(d) 
    
    
    #=============== Find plateau range 
    for j in range(len(data)):
        n = len(data) - 1
        #print(j,n-j)
        if ( (deriv_coin[j] > - tolerance) & (deriv_coin[j] < tolerance) & (low == 0) & (j > 0) ):
            low = data.threshold[j-1]
            #print('low', low)
        if ( (deriv_coin[n-j] > -tolerance) & (deriv_coin[n-j] < tolerance) & (up == 1000) ):
            up = data.threshold[n-j]
            #print('up', up)
    med = (low+up) / 2

    return  deriv_coin, deriv_err, mean_err, low, up, med

In [4]:
#==============================================================
# Read table from excel file (import into a pandas dataframe)
# Use it for 
# - CURVE DI LAVORO (2PMTs)
# - CURVE DI DISCRIMINAZIONE (2 PMTs)
#==============================================================
def read_table_hv(file_name, sheet_name, first_row):
    data = pd.read_excel(file_name, sheetname=sheet_name, header=first_row) ### import table into dataframe 'data'
    
    
    fixed_pmt_name = data['pmt_fixed_name'][0] ### get name of pmt with FIXED HV
    variable_pmt_name = data['pmt_variable_name'][0] ### get name of pmt with VARIABLE HV
    time = data['Time'][0] ### get time window
    HV_fixed = data['HV_fixed'][0] ### get HV value fixed
    
    # Create a new column in the dataframe for the error on HV
    data['HV_err'] = error_on_HV(data['HV'])
    # Create new columns in the dataframe for the error on counts
    data['fixed_counts_err'] = error_on_counts(data['fixed_counts']) ### error on single counts of pmt with FIXED HV
    data['variable_counts_err'] = error_on_counts(data['variable_counts']) ### error on single counts of pmt with VARIABLE HV
    data['coin_err'] = error_on_counts(data['coin']) ### error on coincidence counts    
    
    # Sort dataframe by HV column
    df_ = data.sort_values('HV', ascending=True)
    data = df_.reset_index(drop=True)
    
    
    return data, fixed_pmt_name, variable_pmt_name, time, HV_fixed


def read_table_discr(file_name, sheet_name, first_row):
    data = pd.read_excel(file_name, sheetname=sheet_name, header=first_row) ### import table into dataframe 'data'
    fixed_pmt_name = data['pmt_fixed_name'][0] ### get name of pmt with FIXED HV
    variable_pmt_name = data['pmt_variable_name'][0] ### get name of pmt with VARIABLE HV
    time = data['Time'][0] ### get time window
    threshold_fixed = data['threshold_fixed'][0] ### get HV value fixed
    
    # Create a new column in the dataframe for the error on HV
    data['threshold_err'] = error_on_threshold(data['threshold'])
    # Create new columns in the dataframe for the error on counts
    data['fixed_counts_err'] = error_on_counts(data['fixed_counts']) ### error on single counts of pmt with FIXED HV
    data['variable_counts_err'] = error_on_counts(data['variable_counts']) ### error on single counts of pmt with VARIABLE HV
    data['coin_err'] = error_on_counts(data['coin']) ### error on coincidence counts    
    
    # Sort dataframe by threshold column
    df_ = data.sort_values('threshold', ascending=True)
    data = df_.reset_index(drop=True)
    
    return data, fixed_pmt_name, variable_pmt_name, time, threshold_fixed

In [5]:
### Plot coincidences vs HV
def plot_cdl_main(data):
    lb = variable_pmt_name + ' & ' + fixed_pmt_name
    plt.errorbar(data.HV, data.coin, xerr=data.HV_err, yerr=data.coin_err, fmt='ob', label=lb)
    plt.axvspan(ymin=-1000, ymax=1000, xmin=low, xmax=up, alpha=0.05, color='r')
    lb = 'Plateau center: %.1f V' % med
    plt.axvline(med, ls='-', lw=2, color='r', alpha=0.5, label=lb)
    plt.xlabel('Tensione PMT ' + variable_pmt_name + ' [V]')
    plt.ylabel('Coincidenze in %d s' % time)
    plt.legend(loc='upper left')
    plt.grid()
    return

### Plot slope of coincidences vs HV
def plot_cdl_deriv(data):
    plt.errorbar(data.HV, deriv_coin, deriv_err, fmt='ro')
    #plt.errorbar(data.HV, deriv_coin, fmt='ro')
    plt.ylabel('Derivata del rate [$s^{-1}$]')
    plt.xlabel('Tensione PMT ' + variable_pmt_name + ' [V]')
    mean_err = np.mean(deriv_err)
    plt.axvspan(ymin=-1000, ymax=1000, xmin=low, xmax=up, alpha=0.1, color='r')
    plt.axhspan(ymin=-mean_err, ymax=mean_err, xmin=0, xmax=1000, alpha=0.1)
    plt.axvline(med, ls='-', lw=2, color='r', alpha=0.5)
    plt.axhline(0, ls='--')
    plt.margins(.2)
    return

### Plot single counts of PMT with variable HV vs HV
def plot_cdl_single_variable(data):
    lb = variable_pmt_name
    plt.errorbar(data.HV, data.variable_counts, xerr=data.HV_err, yerr=data.variable_counts_err, fmt='oc', label=lb)
    plt.xlabel('Tensione PMT ' + variable_pmt_name + ' [V]')
    plt.ylabel('Singole in %d s' % time)
    plt.legend(loc='upper left')
    plt.margins(y=.5)
    return

### Plot single counts of PMT with fixed HV vs HV (of variable PMT)
def plot_cdl_single_fixed(data):
    lb = fixed_pmt_name
    plt.errorbar(data.HV, data.fixed_counts, xerr=data.HV_err, yerr=data.fixed_counts_err, fmt='o', color='grey', label=lb)
    plt.xlabel('Tensione PMT ' + variable_pmt_name + ' [V]')
    plt.ylabel('Singole in %d s' % time)
    plt.legend(loc='upper left')
    plt.margins(y=.1)
    return
    
    

#================================= Plot full panel (4 plots) ======================================
def plt_cdl(data):

    # Plot figure with subplots of different sizes
    fig = plt.figure(1)
    fig.set_size_inches(w=12,h=8)
    # set up subplot grid
    rows=4
    columns=5
    gridspec.GridSpec(rows,columns)
    
    # Main title
    title = 'Curva di lavoro - PMT ' + variable_pmt_name + ' (PMT ' + fixed_pmt_name + ' fixed @ %d V)' % HV_fixed
    plt.suptitle(title, fontsize=20, y=1.03)
    
    # large subplot
    plt.subplot2grid((rows,columns), (0,0), rowspan=3, colspan=3)
    plot_cdl_main(data)
    
    # bottom subplot
    plt.subplot2grid((rows,columns), (3,0), rowspan=1, colspan=3)
    plot_cdl_deriv(data)
   
    # top right subplot
    plt.subplot2grid((rows,columns), (0,3), rowspan=2, colspan=2)
    plot_cdl_single_variable(data)
    
    # bottom right subplot
    plt.subplot2grid((rows,columns), (2,3), rowspan=2, colspan=2)
    plot_cdl_single_fixed(data)
    
    plt.tight_layout()
    plt.show()
    
    
    print('Proposed plateau: [%d,%d] V | Central value: %.1f V' % (low,up,med))
    return

In [6]:
### Plot coincidences vs discrimination threshold
def plot_discr_main(data):
    lb = variable_pmt_name + ' & ' + fixed_pmt_name
    plt.errorbar(data.threshold, data.coin, xerr=data.threshold_err, yerr=data.coin_err, fmt='ob', label=lb)
    plt.axvspan(ymin=-1000, ymax=1000, xmin=low, xmax=up, alpha=0.05, color='r')
    lb = 'Plateau center: %.1f V' % med
    plt.axvline(med, ls='-', lw=2, color='r', alpha=0.5, label=lb)
    plt.xlabel('Soglia di discriminazione ' + variable_pmt_name + ' [mV]')
    plt.ylabel('Coincidenze in %d s' % time)
    plt.legend(loc='upper left')
    plt.grid()
    return

### Plot slope of coincidences vs HV
def plot_discr_deriv(data):
    plt.errorbar(data.threshold, deriv_coin, deriv_err, fmt='ro')
    #plt.errorbar(data.threshold, deriv_coin, fmt='ro')
    plt.ylabel('Derivata del rate [$s^{-1}$]')
    plt.xlabel('Soglia di discriminazione ' + variable_pmt_name + ' [mV]')
    mean_err = np.mean(deriv_err)
    plt.axvspan(ymin=-1000, ymax=1000, xmin=low, xmax=up, alpha=0.1, color='r')
    plt.axhspan(ymin=-mean_err, ymax=mean_err, xmin=0, xmax=1000, alpha=0.1)
    plt.axvline(med, ls='-', lw=2, color='r', alpha=0.5)
    plt.axhline(0, ls='--')
    plt.margins(.2)
    return

### Plot single counts of PMT with variable HV vs HV
def plot_discr_single_variable(data):
    lb = variable_pmt_name
    plt.errorbar(data.threshold, data.variable_counts, xerr=data.threshold_err, yerr=data.variable_counts_err, fmt='oc', label=lb)
    plt.xlabel('Soglia di discriminazione ' + variable_pmt_name + ' [mV]')
    plt.ylabel('Singole in %d s' % time)
    plt.legend(loc='upper left')
    plt.margins(y=.5)
    return

### Plot single counts of PMT with fixed HV vs HV (of variable PMT)
def plot_discr_single_fixed(data):
    lb = fixed_pmt_name
    plt.errorbar(data.threshold, data.fixed_counts, xerr=data.threshold_err, yerr=data.fixed_counts_err, fmt='o', color='grey', label=lb)
    plt.xlabel('Soglia di discriminazione ' + variable_pmt_name + ' [mV]')
    plt.ylabel('Singole in %d s' % time)
    plt.legend(loc='upper left')
    plt.margins(y=.1)
    return
    
    

#================================= Plot full panel (4 plots) ======================================
def plt_discr(data):

    # Plot figure with subplots of different sizes
    fig = plt.figure(1)
    fig.set_size_inches(w=12,h=8)
    # set up subplot grid
    rows=4
    columns=5
    gridspec.GridSpec(rows,columns)
    
    # Main title
    title = 'Curva di discriminazione - PMT ' + variable_pmt_name + ' (PMT ' + fixed_pmt_name + ' fixed @ %.0f mV)' % threshold_fixed
    plt.suptitle(title, fontsize=20, y=1.03)
    
    # large subplot
    plt.subplot2grid((rows,columns), (0,0), rowspan=3, colspan=3)
    plot_discr_main(data)
    
    # bottom subplot
    plt.subplot2grid((rows,columns), (3,0), rowspan=1, colspan=3)
    plot_discr_deriv(data)
   
    # top right subplot
    #plt.subplot2grid((rows,columns), (0,3), rowspan=2, colspan=2)
    #plot_discr_single_variable(data)
    
    # bottom right subplot
    #plt.subplot2grid((rows,columns), (2,3), rowspan=2, colspan=2)
    #plot_discr_single_fixed(data)
    
    plt.tight_layout()
    plt.show()
    
    
    print('Proposed plateau: [%d,%d] mV | Central value: %.1f mV' % (low,up,med))
    return